# CLARITY: SemEval 2026 — A100 Training

| Config | Model | Precision | VRAM | Expected F1 |
|--------|-------|-----------|------|-------------|
| `deberta_v3_base.yaml` | DeBERTa-v3-base | auto(bf16) | ~6GB | ~0.48–0.56 |
| `deberta_v3_large.yaml` | DeBERTa-v3-large | bf16 | ~12-14GB | ~0.56–0.65 |

## 1. Setup

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', getattr(props, 'total_mem', 0))
    print(f"VRAM: {vram / 1e9:.1f} GB")
    print(f"bf16: {torch.cuda.is_bf16_supported()}")

In [ ]:
import os, sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    REPO_URL = "https://github.com/wilsebbis/semeval.git"
    if not os.path.exists('/content/semeval'):
        !git clone {REPO_URL} /content/semeval
    else:
        !cd /content/semeval && git pull
    os.chdir('/content/semeval')
    %pip install -e ".[dev]"
    os.environ['HF_HOME'] = '/content/hf_cache'
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir(os.path.join(os.getcwd(), '..'))

src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Working dir: {os.getcwd()}")
from clarity.labels import EVASION_LABELS, validate_labels
validate_labels()
print(f"✓ {len(EVASION_LABELS)} evasion labels")

## 2. Data

In [ ]:
from pathlib import Path
if not Path('data/train.csv').exists():
    !git clone https://huggingface.co/datasets/ailsntua/QEvasion 2>/dev/null || true
    !bash scripts/prepare_data.sh
else:
    print("Data ready.")

In [ ]:
import pandas as pd
train = pd.read_csv('data/train.csv')
dev = pd.read_csv('data/dev.csv')
print(f"Train: {len(train)} | Dev: {len(dev)}")
print(train['evasion_label'].value_counts().to_string())

## 3. Train

In [ ]:
# ── Pick config ──
# CONFIG = "configs/deberta_v3_base.yaml"           # T4/Mac
CONFIG = "configs/deberta_v3_large.yaml"             # A100
# CONFIG = "configs/deberta_v3_large_a100.yaml"      # A100 alt

import yaml
with open(CONFIG) as f:
    cfg = yaml.safe_load(f)
print(f"Config: {CONFIG}")
for k, v in sorted(cfg.items()):
    print(f"  {k}: {v}")

In [ ]:
!python -m clarity.train \
    --config {CONFIG} \
    --data data/train.csv \
    --dev data/dev.csv \
    --task evasion \
    --debug_text

## 4. Evaluate

In [ ]:
import json
output_dir = cfg['output_dir']
mp = Path(output_dir) / 'metrics.json'
if mp.exists():
    m = json.load(open(mp))
    for h in m.get('history', []):
        print(f"  Epoch {h.get('epoch','?')}: F1={h.get('evasion_macro_f1',0):.4f}, loss={h.get('val_loss',0):.4f}")
    print(f"Best F1: {m.get('best_f1', 0):.4f}")
else:
    print('No metrics found.')

## 5. Predict

In [ ]:
!python -m clarity.predict \
    --ckpt {output_dir}/best_model.pt \
    --data data/dev.csv \
    --out submissions/predictions.csv \
    --evaluate

## 6. Ensemble (3 seeds)
Train seeds 43 & 44, then average logits.

In [ ]:
for seed_cfg in ['configs/deberta_v3_large_seed43.yaml',
                 'configs/deberta_v3_large_seed44.yaml']:
    print(f"\n{'='*60}\nTraining: {seed_cfg}\n{'='*60}")
    !python -m clarity.train --config {seed_cfg} --data data/train.csv --dev data/dev.csv --task evasion

In [ ]:
!python -m clarity.ensemble \
    --ckpts \
        checkpoints/deberta_v3_large/best_model.pt \
        checkpoints/deberta_v3_large_seed43/best_model.pt \
        checkpoints/deberta_v3_large_seed44/best_model.pt \
    --data data/dev.csv \
    --out submissions/ensemble_predictions.csv \
    --task evasion \
    --evaluate

## 7. Download

In [ ]:
if IN_COLAB:
    from google.colab import files
    files.download('submissions/predictions.csv')
    if Path('submissions/ensemble_predictions.csv').exists():
        files.download('submissions/ensemble_predictions.csv')
else:
    print(f"Results: {os.path.abspath('submissions/')}")